In [1]:
import pandas as pd
import datetime
import datetime
import re
import lxml
from bs4 import BeautifulSoup
import json
from calendar import monthrange
import numpy as np

In [2]:
print(pd.__version__)

0.22.0


In [3]:
#date_str = datetime.datetime.now().strftime("%Y-%m-%d")
date_str = '2018-02-22a'

js = pd.read_json('/Users/taj/GitHub/scraping/chrome_stayz_calendar/WebData/stayz_calendar_' + date_str + '.json')
#                 ,compression='zip')

js.reset_index(drop=True)
js.set_index('property_id')

# Change values of -1.0 into NaN for stats analysis
#js.loc[js['review_count'] == -1.0, 'review_count'] = np.nan
#js.loc[js['review_value'] == -1.0, 'review_value'] = np.nan

,calendar,ext_at,review_count,review_value
property_id,,,,
9153367,"<thead><tr><th scope=""col""><span title=""Monday...",2018-02-23 07:00:14,38,4.7
9076897,"<thead><tr><th scope=""col""><span title=""Monday...",2018-02-23 07:00:16,28,4.6
9123458,"<thead><tr><th scope=""col""><span title=""Monday...",2018-02-23 07:00:19,71,4.9
9114125,"<thead><tr><th scope=""col""><span title=""Monday...",2018-02-23 07:00:20,27,4.4
9063683,"<thead><tr><th scope=""col""><span title=""Monday...",2018-02-23 07:00:23,30,4.9
9161490,"<thead><tr><th scope=""col""><span title=""Monday...",2018-02-23 07:00:26,20,4.9
9195589,"<thead><tr><th scope=""col""><span title=""Monday...",2018-02-23 07:00:27,4,4.8
9167523,"<thead><tr><th scope=""col""><span title=""Monday...",2018-02-23 07:00:30,19,5.0
9095729,"<thead><tr><th scope=""col""><span title=""Monday...",2018-02-23 07:00:31,22,4.8


In [4]:
print(js.head())

                                            calendar              ext_at  \
0  <thead><tr><th scope="col"><span title="Monday... 2018-02-23 07:00:14   
1  <thead><tr><th scope="col"><span title="Monday... 2018-02-23 07:00:16   
2  <thead><tr><th scope="col"><span title="Monday... 2018-02-23 07:00:19   
3  <thead><tr><th scope="col"><span title="Monday... 2018-02-23 07:00:20   
4  <thead><tr><th scope="col"><span title="Monday... 2018-02-23 07:00:23   

  property_id  review_count  review_value  
0     9153367            38           4.7  
1     9076897            28           4.6  
2     9123458            71           4.9  
3     9114125            27           4.4  
4     9063683            30           4.9  


In [5]:
d = datetime.date.today()
mr = monthrange(d.year, d.month )

cur_month = d.month
cur_year = d.year


first_page = True



fp = open('/Users/taj/GitHub/scraping/chrome_stayz_calendar/WebData/stayz_processed_calendar_' + date_str + '.json', 'w')


for index, row in js.iterrows():   
    
    all_months = {}
    
    cur_month = d.month

    raw_cal = "<html><body><table>" + row['calendar'] + "</table></body></html>"
    
    
    soup = BeautifulSoup(raw_cal,'lxml') # Parse the HTML as a string
    soup_tables = soup.findAll('tbody')# Grab the first table
    
    
    month_tb_count = 1

    for month_tb in soup_tables:
 
        cells = month_tb.findAll("td")
        
        for td in cells:
            try:
                day_val = int(td.text)

                date_full = datetime.date(cur_year,cur_month,day_val)

                m1 = None
                m2 = None
                m3 = None
                m4 = None

                #----------------------------------------------------------
                # Check if DEP and ARR can both be on the same day
                # for consecutive bookings?

                # If its an unavailable date then mark: 
                m1 = re.search('c-calendar--unavailable', str(td))

                if m1 is not None:
                    all_months[str(date_full)] = 'UVL' # Unavailable day

                # If its an unavailable date then mark: 
                m2 = re.search('c-calendar--arrival-day', str(td))

                if m2 is not None:
                    all_months[str(date_full)] = 'ARR' # Arrival day

                # If its an unavailable date then mark: 
                m3 = re.search('c-calendar--departure-day', str(td))

                if m3 is not None:
                    all_months[str(date_full)] = 'DEP' # Departure day

                m4 = re.search('c-calendar--available', str(td))
                if m4 is not None:
                    all_months[str(date_full)] = 'AVL' # Available day


            except ValueError:
                day_val = None

        # Move to the next month
        cur_month += 1
        month_tb_count += 1

    
    cal_details = {}
    cal_details['property_id'] = row['property_id']
    cal_details['calendar'] = all_months
    
    if first_page is True:
        fp.write('[\n')
        first_page = False
    else:
        fp.write('\n,')
        
    # Write the data to JSON
    json.dump(cal_details, fp)


    
# Close out the JSON format
fp.write('\n]')    

# Tidy up
fp.close()

In [6]:
# Open the Processed Calendar file
p = pd.read_json('/Users/taj/GitHub/scraping/chrome_stayz_calendar/WebData/stayz_processed_calendar_' + date_str + '.json')
p.head()

,calendar,property_id
0,"{'2018-03-01': 'AVL', '2018-03-02': 'AVL', '20...",9153367
1,"{'2018-03-01': 'AVL', '2018-03-02': 'AVL', '20...",9076897
2,"{'2018-03-01': 'AVL', '2018-03-02': 'AVL', '20...",9123458
3,"{'2018-03-01': 'AVL', '2018-03-02': 'AVL', '20...",9114125
4,"{'2018-03-01': 'AVL', '2018-03-02': 'AVL', '20...",9063683


In [7]:
# Open the file for writing out the bookings details:
fp = open('/Users/taj/GitHub/scraping/chrome_stayz_calendar/WebData/stayz_bookings_' + date_str + '.json', 'w')

first_page = True

# Iterate over each entry in the Processed Calendar file:
for index, row in p.iterrows():  
    
    c = row['calendar']
    #print(c)
    
    pid = row['property_id']
    
    
    #c2 = c.to_dict()[0]
    #type(c2)


    days_count = 0
    avl_count = 0
    dep_count = 0
    arr_count = 0
    uvl_count = 0

    booking_count = 0


    dates = list(c.keys())

    min_dateIndex = 0
    max_dateIndex = len(dates)

    #print("Number of days: " + str(max_dateIndex))
    
    while (min_dateIndex < max_dateIndex):


        date = dates[min_dateIndex]
        status = c[date]


        # Count the total number of days
        days_count += 1

        if( status == 'AVL'):
            avl_count += 1

        if( status == 'ARR'):
            arr_count += 1
            # Iterate while the days are Unavailble until we find a Departure

            # Keep the arrival date:
            date_arr = date

            # Reset the count for this booking
            booking_days = 0

            # Move to the next day.
            # Breaks if they arrive on the last day of the 6th month!!!
            min_dateIndex += 1
            
            if(min_dateIndex < max_dateIndex): 
                date = dates[min_dateIndex]
                status = c[date]
            else:
                date = None
                status = None
            

            while(( status != 'DEP') & (min_dateIndex < max_dateIndex)):
                date = dates[min_dateIndex]
                status = c[date]

                # Departure date doesnt count as a booked date, but as an available date
                booking_days += 1

                days_count += 1

                min_dateIndex += 1
                
                # on the last day of the month it breaks the index...
                #date = dates[min_dateIndex]
                #status = c[date]



            # Add in the last day
            avl_count += 1
            days_count += 1

            # Get the departure day details??
            
            if(min_dateIndex >= max_dateIndex): 
                # Booking runs over the end of the month into the 7th month, which we dont track
                # Ignore this booking or just track to the end of the month?
                date_dep = None
            else:
                date_dep = date

            # Track the total bookings.
            booking_count += booking_days


            # Show the booking details
            #print("{0} -- Arrive: {1} Depart: {2} Total days: {3}".format(pid,date_arr,date_dep,booking_days))

            booking_detail = {
                'property_id': pid,
                'arr_dt': date_arr,
                'dep_dt': date_dep,
                'book_days': booking_days
            }

            if first_page is True:
                fp.write('[\n')
                first_page = False
            else:
                fp.write('\n,')

            json.dump(booking_detail, fp)



        #if( status == 'DEP'):
        #    dep_count += 1            

        if( status == 'UVL'):
            uvl_count += 1

        min_dateIndex += 1

# Close off the JSON
fp.write(']')

#print("Total days: " + str(days_count))
#print("Available days: " + str(avl_count))
#print("Departure days: " + str(dep_count))
#print("Arrival days: " + str(arr_count))
#print("Booked days: " + str(booking_count))
#print("Unavailable days: " + str(uvl_count))
#print("Total check: " + str(avl_count + booking_count + uvl_count))

# Tidy up file handles
fp.close()

In [8]:
b = pd.read_json('/Users/taj/GitHub/scraping/chrome_stayz_calendar/WebData/stayz_bookings_' + date_str + '.json'
                ,convert_dates=['arr_dt','dep_dt'])

b.set_index('property_id',inplace=True)

#b.head()
#bin_values = np.arange(start=0, stop=600, step=10)
#b['book_days'].plot(kind='hist', bins=bin_values, figsize=[12,6], alpha=.4) # alpha for transparency